# Main Library

In [18]:
# Reading Data & Visualization & Preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

# Build Model
from torch import Tensor
from torch.nn import Module, ReLU, Softmax, Linear, CrossEntropyLoss, Sigmoid, BCELoss
from torch.optim import Adam, NAdam, SGD 
from torch.utils.data import DataLoader, Dataset, random_split

In [19]:
# import torch
# torch.cuda.is_available = lambda : False
# # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
class Fetch_Data():
    def __init__(self, path):
        data = pd.read_csv(path)
        data = data.drop(columns=["id", "Unnamed: 32"])
        self.x = data.drop(columns="diagnosis", axis=1).values
        self.x = self.x.astype("float32")
        
        self.y = data["diagnosis"].values
        self.y = OneHotEncoder().fit_transform(self.y.reshape(-1, 1)).toarray()
        self.y = self.y.astype("float32")
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def split_data(self, ratio=0.5):
        size1 = round(len(self.x) * ratio)
        size2 = len(self.x) - size1

        return random_split(self, [size1, size2])
    
def Get_data(path, ratio=0.5):
    obj = Fetch_Data(path)
    
    train, test = obj.split_data(ratio)
    
    train = DataLoader(train, shuffle=True, batch_size=40)
    
    test = DataLoader(test, shuffle=False, batch_size=1000)
    
    return train, test
    

In [21]:
class NeuralNetwork(Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        
        self.layer1 = Linear(input_size, 128)
        self.activation1 = ReLU()
                
        self.layer2 = Linear(128, 32)
        self.activation2 = ReLU()
        
        self.layer3 = Linear(32, 8)
        self.activation3 = ReLU()
        
        self.layer4 = Linear(8, output_size)
        self.activation4 = Sigmoid()
        
        
    def forward(self, data):
        data = self.layer1(data)
        data = self.activation1(data)
        
        data = self.layer2(data)
        data = self.activation2(data)
        
        data = self.layer3(data)
        data = self.activation3(data)
        
        data = self.layer4(data)
        data = self.activation4(data)
        
        return data
    
    
def training_model(train_data, model):
    loss_func = BCELoss()
    optamizer = Adam(model.parameters(), lr=0.01)
    
    for opochs in range(100):
        
        for (x, y) in train_data:
            optamizer.zero_grad()
            
            result = model(x)
            loss = loss_func(result, y)
            
            loss.backward()
            optamizer.step()
            
            

def Evaluation(test_Data, model):
    accumlate_result = []
    accumlate_y = []
    
    for (x, y) in test_Data:
        result = model(x)
        
        result = result.detach().numpy()
        y = y.detach().numpy()
        result = result.round()
        
        accumlate_result.append(result)
        accumlate_y.append(y)
        
    accumlate_result = np.vstack(accumlate_result)
    accumlate_y = np.vstack(accumlate_y)
    
    print("The Accuracy Score is ", round(accuracy_score(accumlate_y, accumlate_result), 2) * 100, "%")

In [24]:
train, test = Get_data(r"D:\Courses language programming\5_Machine Learning\Dataset For Machine Learning\Breast Cancer\data.csv",
                       ratio=0.7)
input_size = len(train.dataset[0][0])
output_size = len(train.dataset[0][1])

model = NeuralNetwork(input_size, output_size)

training_model(train, model)

Evaluation(test, model)

The Accuracy Score is  96.0 %


In [25]:
Evaluation(train, model)

The Accuracy Score is  93.0 %
